In [1]:
import sidmhalo
from sidmhalo.classes import load_profile
import os

import numpy as np

# test the saving and loading functionality 

In [2]:
rm = 30 # radius of isothermal core, kpc
M200 = 1e12 # virial mass, Msun
R200 = 200 # virial radius, kpc
c = 10 # concentration, dimensionless
q_sph = 1 # spherical axis ratio, dimensionless
Phi_b = None # baryon potential, Msun/kpc^2

profile = sidmhalo.gen.squashed(rm, M200, c, q0=q_sph, Phi_b=Phi_b)

In [3]:
filename = 'data/test_profile.npz'

profile.save(filename)

In [4]:
# loading the profile
lprofile = load_profile(filename)

In [5]:
lprofile.rho_sph_avg(100)

36731.99447813377

In [7]:
type(lprofile)

sidmhalo.classes.profile

In [8]:
load_profile.r1

AttributeError: 'function' object has no attribute 'r1'

In [17]:
data = np.load(filename, allow_pickle=True)
data['profile_data']

array({'class_type': 'profile', 'inner': {'class_type': 'isothermal_profile', 'y': array([0.00000000e+00, 0.00000000e+00, 2.93360178e-07, 1.76016101e-09,
       3.21811948e-07, 2.02233875e-09, 3.53023135e-07, 2.32356812e-09,
       3.87261365e-07, 2.66966590e-09, 4.24820216e-07, 3.06731529e-09,
       4.66021742e-07, 3.52419495e-09, 5.11219230e-07, 4.04912729e-09,
       5.60800233e-07, 4.65224881e-09, 6.15189888e-07, 5.34520588e-09,
       6.74854567e-07, 6.14137958e-09, 7.40305872e-07, 7.05614414e-09,
       8.12105023e-07, 8.10716377e-09, 8.90867671e-07, 9.31473380e-09,
       9.77269176e-07, 1.07021726e-08, 1.07205040e-06, 1.22962717e-08,
       1.17602405e-06, 1.41278134e-08, 1.29008167e-06, 1.62321650e-08,
       1.41520126e-06, 1.86499619e-08, 1.55245566e-06, 2.14278918e-08,
       1.70302179e-06, 2.46195971e-08, 1.86819069e-06, 2.82867100e-08,
       2.04937863e-06, 3.25000427e-08, 2.24813921e-06, 3.73409553e-08,
       2.46617674e-06, 4.29029264e-08, 2.70536081e-06, 4.92933582

In [15]:
print(data['M200'], data['c'], data['q0'], data['AC_prescription'], data['r_list'])

1000000000000.0 10 1 None [0.00000000e+00 3.00000000e-03 3.14211269e-03 3.29095739e-03
 3.44685300e-03 3.61013352e-03 3.78114879e-03 3.96026520e-03
 4.14786652e-03 4.34435468e-03 4.55015067e-03 4.76569539e-03
 4.99145066e-03 5.22790016e-03 5.47555048e-03 5.73493223e-03
 6.00660112e-03 6.29113920e-03 6.58915612e-03 6.90129036e-03
 7.22821068e-03 7.57061751e-03 7.92924446e-03 8.30485989e-03
 8.69826856e-03 9.11031335e-03 9.54187708e-03 9.99388436e-03
 1.04673036e-02 1.09631492e-02 1.14824834e-02 1.20264190e-02
 1.25961213e-02 1.31928108e-02 1.38177661e-02 1.44723261e-02
 1.51578932e-02 1.58759362e-02 1.66279936e-02 1.74156765e-02
 1.82406728e-02 1.91047498e-02 2.00097590e-02 2.09576392e-02
 2.19504214e-02 2.29902326e-02 2.40793006e-02 2.52199587e-02
 2.64146507e-02 2.76659365e-02 2.89764967e-02 3.03491394e-02
 3.17868054e-02 3.32925749e-02 3.48696741e-02 3.65214818e-02
 3.82515372e-02 4.00635469e-02 4.19613931e-02 4.39491419e-02
 4.60310523e-02 4.82115845e-02 5.04954106e-02 5.28874235e-0